In [1]:
import pickle
import time
import pandas as pd
import numpy as np
from itertools import combinations
from collections import defaultdict
import csv

In [2]:
clst_file = "../ppi_ml/results/ppi_clustering/final_cmplx_annots_011823.csv"
apms_pkl = "../ppi_ml/data/apms/apms_scores_dict.pkl"
apms_fmat_file = "../ppi_ml/data/apms/featmat_humap"
outdir = "../ppi_ml/data/apms/og_scores/"

In [3]:
with open(apms_pkl, 'rb') as handle:
    apms_dict = pickle.load(handle)

In [4]:
df = pd.read_csv(clst_file)
ogs = df['ID'].tolist()
len(ogs)

2778

In [5]:
cmplx_groups = df.groupby('granulated_cmplx_name')
cmplx_groups.groups.keys()

dict_keys(['(ER)-localized multiprotein complex 1', '(ER)-localized multiprotein complex 2', '(ER)-localized multiprotein complex 3', '19S Proteasome', '2-oxoglutarate dehydrogenase complex', '20S Proteasome', '26S Proteasome', '3-phosphoinositide-dependent protein kinase', '37S mitochondrial small ribosomal subunit', '3M complex', '43S preinitiation complex', "5'-deoxynucleotidase", '7SK RNP complex', '90S preribosome', 'ABC transporter complex', 'ABCG sterol transporter complex', 'ABITRAM-PUS10 cell death complex', 'ADP-ribosylation factor GTPase-activating protein', 'AGO/TrxG complex', 'AMP deaminase', 'AMPA glutamate receptor complex', 'AMPA glutamate receptor complex 1', 'AMPA glutamate receptor complex 2', 'AMPK complex', 'ASAP complex', 'ATAC complex', 'ATR-ATRIP DNA damage-sensing kinase complex', 'Acetolactate synthase complex 1', 'Acetolactate synthase complex 2', 'Adaptor complex AP-1', 'Adaptor complex AP-2', 'Adaptor complex AP-3', 'Afadin-beta-catenin complex', 'Aminoacyl

In [6]:
#for c, p in cmplx_groups:
    #print(c, '\n', p['ID'].tolist())

In [7]:
pair_dict = dict()
issues = []
for cmplx, df in cmplx_groups:
    ogs = df['ID'].tolist()
    if len(ogs) > 1:
        for i in ogs:
            self_interaction = frozenset({i, i})
        pairs = [frozenset({i, j}) for i,j in list(combinations(ogs, 2))]
        pairs.append(self_interaction)
        if cmplx not in pair_dict.keys():
            pair_dict.update({cmplx: pairs})
        else:
            issues.append(cmplx)
            pair_dict.pop(cmplx)

In [8]:
all_pairs = []
for c, p in pair_dict.items():
    for pair in p:
        if pair not in all_pairs:
            all_pairs.append(pair)
len(all_pairs)

19367

In [9]:
out_dict = dict()
for k, v in pair_dict.items():
    vals = []
    for pair in v:
        scores = apms_dict.get(pair)
        # oh man, this is ugly
        # ... but it works ¯\_(ツ)_/¯
        if scores:
            for score_list in scores:
                for i, s in enumerate(score_list):
                    if s == '':
                        score_list[i] = np.nan
        out_dict[pair] = scores
    if not vals:
        continue

In [11]:
with open('/stor/work/Marcotte/project/rmcox/leca/ppi_ml/data/apms/clustered_og_scores_dict.pkl', 'wb') as handle:
    pickle.dump(out_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [12]:
with open(apms_fmat_file, 'r') as f:
    cols = f.readline().strip('\n').split(',')
    score_cols = cols[1:]

In [14]:
df = pd.DataFrame([out_dict.keys(), out_dict.values()]).T
df[['ID1', 'ID2']] = pd.DataFrame(df[0].tolist(), index=df.index)

In [15]:
df

,0,1,ID1,ID2
0,"(KOG0730, KOG0020)","[[106.1875, 0.0, nan, nan, nan, nan, nan, nan,...",KOG0730,KOG0020
1,"(KOG4563, KOG0020)","[[0.0, 0.0, nan, nan, nan, nan, nan, nan, nan,...",KOG4563,KOG0020
2,"(KOG4563, KOG0730)","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...",KOG4563,KOG0730
3,(KOG4563),"[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...",KOG4563,None
4,"(KOG0190, KOG3544)","[[nan, nan, nan, nan, nan, nan, nan, 5.0267844...",KOG0190,KOG3544
...,...,...,...,...
19362,"(KOG0115, KOG4380)","[[0.0, 0.0, nan, nan, nan, nan, nan, nan, nan,...",KOG0115,KOG4380
19363,"(KOG1914, KOG3833)","[[0.0, 0.0, nan, nan, nan, nan, nan, nan, nan,...",KOG1914,KOG3833
19364,"(KOG4380, KOG3833)","[[0.0, 0.0, nan, nan, nan, nan, nan, nan, nan,...",KOG4380,KOG3833
19365,"(KOG1914, KOG4380)","[[0.0, 0.0, nan, nan, nan, nan, nan, nan, nan,...",KOG1914,KOG4380


In [16]:
df2 = df[1].apply(pd.Series, dtype=float)

In [17]:
df_scores = pd.DataFrame(df2[0].to_list(), columns=score_cols)

In [18]:
df_final = df[['ID1', 'ID2']].join(df_scores)

In [19]:
df_final

,ID1,ID2,ext_Dm_guru,ext_Hs_malo,entropy_orig9k,zscore_orig9k,nwdscore_orig9k,plate_zscore_orig9k,uPeps_orig9k,neg_ln_pval,...,neg_ln_pval_treiber_hygeo_gt4,pair_count_treiber_hygeo_gt4,neg_ln_pval_youn_hygeo,pair_count_youn_hygeo,neg_ln_pval_youn_hygeo_gt2,pair_count_youn_hygeo_gt2,neg_ln_pval_youn_hygeo_gt4,pair_count_youn_hygeo_gt4,neg_ln_pval_treiber_hygeo_gt2,pair_count_treiber_hygeo_gt2
0,KOG0730,KOG0020,106.1875,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,321.362170611,185.0,1.7807923174319198,21.0,4.2086402670345,8.5,0.838603306625,2.0,237.95390510428,141.0
1,KOG4563,KOG0020,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.79477044991,82.0,0.574994780399,45.0,1.43854062237,12.0,0.858966105654,3.0
2,KOG4563,KOG0730,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.415840928281,2.0,NaN,NaN,NaN,NaN,NaN,NaN
3,KOG4563,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,KOG0190,KOG3544,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.026784491335,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19362,KOG0115,KOG4380,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,-2.4487079031100004e-12,2.0,NaN,NaN,NaN,NaN,NaN,NaN,-2.22044604925e-16,4.0
19363,KOG1914,KOG3833,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,1.09752412691,1.0,0.229261697311,15.0,0.300876679894,8.0,0.0194486627932,1.0,0.865448414471,3.0
19364,KOG4380,KOG3833,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,-5.39124300758e-13,1.0,NaN,NaN,NaN,NaN,NaN,NaN,-2.22044604925e-16,4.0
19365,KOG1914,KOG4380,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
def match_cmplx(x, my_dict):
    for cmplx, pair_list in my_dict.items():
        if any(pair == x for pair in pair_list):
            return(cmplx)

In [21]:
df_final['cmplx'] = [match_cmplx(frozenset({i, j}), pair_dict) for i, j in zip(df['ID1'], df['ID2'])]
df_final

,ID1,ID2,ext_Dm_guru,ext_Hs_malo,entropy_orig9k,zscore_orig9k,nwdscore_orig9k,plate_zscore_orig9k,uPeps_orig9k,neg_ln_pval,...,pair_count_treiber_hygeo_gt4,neg_ln_pval_youn_hygeo,pair_count_youn_hygeo,neg_ln_pval_youn_hygeo_gt2,pair_count_youn_hygeo_gt2,neg_ln_pval_youn_hygeo_gt4,pair_count_youn_hygeo_gt4,neg_ln_pval_treiber_hygeo_gt2,pair_count_treiber_hygeo_gt2,cmplx
0,KOG0730,KOG0020,106.1875,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,185.0,1.7807923174319198,21.0,4.2086402670345,8.5,0.838603306625,2.0,237.95390510428,141.0,(ER)-localized multiprotein complex 1
1,KOG4563,KOG0020,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.79477044991,82.0,0.574994780399,45.0,1.43854062237,12.0,0.858966105654,3.0,(ER)-localized multiprotein complex 1
2,KOG4563,KOG0730,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.415840928281,2.0,NaN,NaN,NaN,NaN,NaN,NaN,(ER)-localized multiprotein complex 1
3,KOG4563,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
4,KOG0190,KOG3544,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.026784491335,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(ER)-localized multiprotein complex 2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19362,KOG0115,KOG4380,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,NaN,NaN,NaN,NaN,NaN,NaN,-2.22044604925e-16,4.0,tRNA-splicing ligase complex
19363,KOG1914,KOG3833,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,0.229261697311,15.0,0.300876679894,8.0,0.0194486627932,1.0,0.865448414471,3.0,tRNA-splicing ligase complex
19364,KOG4380,KOG3833,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,-2.22044604925e-16,4.0,tRNA-splicing ligase complex
19365,KOG1914,KOG4380,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tRNA-splicing ligase complex


In [34]:
df_final.to_csv('../ppi_ml/data/apms/clustered_og_scores.csv', index=False)

In [38]:
top_feats = ['pair_count_cilium_hygeo', 'neg_ln_pval_youn_hygeo_gt2', 'neg_ln_pval_youn_hygeo', 'ave_apsm', 'neg_ln_pval_treiber_hygeo_gt2'] # top 10 feat importance
select_feats = ['neg_ln_pval_bioplex2_Z2', 'neg_ln_pval_cilium_hygeo', 'neg_ln_pval_boldt_apms_hygeo',
          'ave_apsm', 'neg_ln_pval_treiber_hygeo_gt2'] # more distributed across different data sets

In [39]:
scores_subset = df_final[top_feats]
df_subset = df_final[['ID1', 'ID2', 'cmplx']].join(scores_subset)
df_subset.reset_index(drop=True)

,ID1,ID2,cmplx,neg_ln_pval_bioplex2_Z2,neg_ln_pval_cilium_hygeo,neg_ln_pval_boldt_apms_hygeo,ave_apsm,neg_ln_pval_treiber_hygeo_gt2
0,KOG0730,KOG0020,(ER)-localized multiprotein complex 1,6.177897327198999,6.05435897166,7.45725773959,6.5,237.95390510428
1,KOG4563,KOG0020,(ER)-localized multiprotein complex 1,0.891453940996,0.688591057219,18.1826657527,15.0,0.858966105654
2,KOG4563,KOG0730,(ER)-localized multiprotein complex 1,NaN,NaN,NaN,NaN,NaN
3,KOG4563,None,None,NaN,NaN,NaN,650.0,NaN
4,KOG0190,KOG3544,(ER)-localized multiprotein complex 2,16.395327781112,3.104226303415,NaN,31.142857142857142,NaN
...,...,...,...,...,...,...,...,...
19362,KOG0115,KOG4380,tRNA-splicing ligase complex,57.2519106281,0.0,NaN,8.0,-2.22044604925e-16
19363,KOG1914,KOG3833,tRNA-splicing ligase complex,NaN,0.361013345537,NaN,NaN,0.865448414471
19364,KOG4380,KOG3833,tRNA-splicing ligase complex,138.798905149,9.86504895351,4.28128506487,NaN,-2.22044604925e-16
19365,KOG1914,KOG4380,tRNA-splicing ligase complex,NaN,0.500775287912,NaN,NaN,NaN


In [40]:
df_subset.to_csv('../ppi_ml/data/apms/clustered_og_scores_subset.csv', index=False)

These scores are shockingly low & sparse among pairs in the final clustered data set. Trying some things to make sure nothing went wrong during the mapping script:

In [26]:
test_pair = set(['KOG2604', 'KOG4673'])
print(f'Evaluating out_dict for {test_pair}...')
for pair, score_list in out_dict.items():
    if test_pair == pair:
        print(pair, ':')
        print(score_list)

print()
print(f'Evaluating apms_dict for {test_pair} ...')
for pair, score_list in apms_dict.items():
    if test_pair == pair:
        print(pair, ':')
        print(score_list)

Evaluating out_dict for {'KOG4673', 'KOG2604'}...
frozenset({'KOG4673', 'KOG2604'}) :
[[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, '5.10657685349', '2.0', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]]

Evaluating apms_dict for {'KOG4673', 'KOG2604'} ...
frozenset({'KOG4673', 'KOG2604'}) :
[[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, '5.10657685349', '2.0', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]]


Evaluating out_dict for {'KOG2604', 'KOG4673'}...
frozenset({'KOG2604', 'KOG4673'}) :
[['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '5.10657685349', '2', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']]

Evaluating apms_dict for {'KOG2604', 'KOG4673'} ...
frozenset({'KOG2604', 'KOG4673'}) :
[['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '5.10657685349', '2', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']]

Featmap vals for KOG2604 + KOG4673:
KOG2604 KOG4673,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5.10657685349,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

Hmmmmm this looks good... let's try one more: KOG0178 + KOG0176 (PSMA4 + PSMA5)

In [27]:
test_pair = set(['KOG0178', 'KOG0176'])
print(f'Evaluating out_dict for {test_pair}...')
for pair, score_list in out_dict.items():
    if test_pair == pair:
        print(pair, ':')
        print(score_list)

print()
print(f'Evaluating apms_dict for {test_pair} ...')
for pair, score_list in apms_dict.items():
    if test_pair == pair:
        print(pair, ':')
        print(score_list)

Evaluating out_dict for {'KOG0176', 'KOG0178'}...
frozenset({'KOG0176', 'KOG0178'}) :
[['708.5', '10.0', nan, nan, nan, nan, nan, '32.7158829408', '5.0', nan, nan, '10.6058422138', '2.0', nan, nan, nan, nan, nan, nan, '52.61376206', '8.0', '71.2756412064', '14.0', nan, nan, nan, nan, nan, nan, '2.50396', '0.0', '0.0', '2.50396', '25.0586119655', '22.0', '25.4795048226', '21.0', '5.12098555801', '1.0', '4.0797574513', '5.0', nan, nan, nan, nan, '29.6439880767', '9.0']]

Evaluating apms_dict for {'KOG0176', 'KOG0178'} ...
frozenset({'KOG0176', 'KOG0178'}) :
[['708.5', '10.0', nan, nan, nan, nan, nan, '32.7158829408', '5.0', nan, nan, '10.6058422138', '2.0', nan, nan, nan, nan, nan, nan, '52.61376206', '8.0', '71.2756412064', '14.0', nan, nan, nan, nan, nan, nan, '2.50396', '0.0', '0.0', '2.50396', '25.0586119655', '22.0', '25.4795048226', '21.0', '5.12098555801', '1.0', '4.0797574513', '5.0', nan, nan, nan, nan, '29.6439880767', '9.0']]


In [28]:
x = 'KOG2604'
print()
print(f'Evaluating apms_dict for {x} ...')
for pair, score_list in apms_dict.items():
    if any(x in p for p in pair):
        print(pair, ':')
        print(score_list)


Evaluating apms_dict for KOG2604 ...
frozenset({'ENOG502QPJP', 'KOG2604'}) :
[['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '0.726939757776', '1.0', '1.28747740443', '1.0', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']]
frozenset({'ENOG502QPKK', 'KOG2604'}) :
[['', '', '', '', '', '', '', '', '', '', '', '', '', '2.0', '0.1', '1.98', '3.23203', '1.0', '3.0', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']]
frozenset({'ENOG502QPKN', 'KOG2604'}) :
[['', '', '0.773226674288', '27.72', '4.62', '8.48528137424', '9.0', '6.25036126813', '1.0', '', '', '', '', '4.5', '4.43', '2.37', '4.404263', '0.868653', '7.5', '5.27896666437', '1.0', '1.23271116106', '1.0', '3.97412534195', '4.0', '4.62350114267', '3.0', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']]
frozenset({'ENOG502QPKV', 'KOG2604'}) :
[['', '', '', '', '', ''

In [29]:
x = 'KOG0176'
print()
print(f'Evaluating apms_dict for {x} ...')
for pair, score_list in apms_dict.items():
    if any(x in p for p in pair):
        print(pair, ':')
        print(score_list)


Evaluating apms_dict for KOG0176 ...
frozenset({'KOG0176', 'ENOG502QPJP'}) :
[['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '0.795135136973', '1.0', '', '', '', '', '', '']]
frozenset({'KOG0176', 'ENOG502QPJR'}) :
[['', '', '', '', '', '', '', '', '', '', '', '', '', '3.0', '0.03', '0.15', '0.302358', '0.979869', '4.0', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']]
frozenset({'ENOG502QPKE', 'KOG0176'}) :
[['', '', '', '', '', '', '', '', '', '', '', '', '', '3.0', '0.03', '0.15', '0.892083', '0.940286', '4.0', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']]
frozenset({'ENOG502QPKJ', 'KOG0176'}) :
[['', '', '', '', '', '', '', '', '', '', '', '', '', '4.0', '0.03', '0.51', '1.42144', '0.988699', '4.0', '', '', '1.24462096095', '1.0', '', '', '', '